In [1]:
from gurobipy import GRB
import gurobipy as gb

In [2]:
import pandas as pd

In [5]:
# Create model
model = gb.Model("FIBA")

# Read dataset
df = pd.read_csv('/Users/frieda/Desktop/BasketballPlayers.csv', index_col='Number')

# Create decision variables
x = model.addVars(150, vtype=gb.GRB.BINARY, name="Player")

# Set the objective function
model.setObjective(gb.quicksum(df.iloc[i, 1:8].sum() * x[i] for i in range(150)), gb.GRB.MAXIMIZE)

# Add constraints

# Capacity constraint
model.addConstr(gb.quicksum(x[i] for i in range(150)) == 21, "Total Players")

# Position constraints
guard_indices = df[(df['Position'] == 'G') | (df['Position'] == 'G/F')].index.tolist()
model.addConstr(gb.quicksum(x[i-1] for i in guard_indices) >= 0.3 * 21, "Guard Position")

forward_center_indices = df[(df['Position'] == 'F') | (df['Position'] == 'C') | (df['Position'] == 'F/C')].index.tolist()
model.addConstr(gb.quicksum(x[i-1] for i in forward_center_indices) >= 0.4 * 21, "Forward/Center Position")

# Average score constraints
for j in range(7):
    model.addConstr((gb.quicksum(x[i] * df.iloc[i, j+1] for i in range(150)) / 21) >= 2.05, f"Average Score {j}")

# Constraint: if any 20-24, not all 72-78
for i in range(19, 24):
    model.addConstr(x[i] <= 1 - gb.quicksum(x[j] for j in range(71, 78)), f"If any 20-24, not all 72-78")

# Constraint: if any 105-114, at least one 45-49 and at least one 60-69
for i in range(104, 114):
    model.addConstr(x[i] <= gb.quicksum(x[j] for j in range(44, 49)), f"If any 105-114, at least 45-49")
    model.addConstr(x[i] <= gb.quicksum(x[j] for j in range(64, 69)), f"If any 105-114, at least 65-69")

# At least one player from every 10 players
for i in range(14):
    model.addConstr(gb.quicksum(x[i*10 + j] for j in range(10)) >= 1, f"At least one player from group {i}")

# Optimize the model
model.optimize()

# Print the objective and decision variables
model.printAttr('X')

# Print the model status
print("Model Status: ", model.status)

# Count and print the number of guards invited
selected = [i for i, var in enumerate(model.getVars()) if var.x != 0]
guards_count = sum(1 for i in selected if df.iloc[i]['Position'] in ['G', 'G/F'])
print(f'Number of G and G/F: {guards_count}')


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 20.2.0 20C69)

CPU model: Intel(R) Core(TM) i5-5257U CPU @ 2.70GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 49 rows, 150 columns and 1650 nonzeros
Model fingerprint: 0xbc35fceb
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 49 rows, 149 columns, 1298 nonzeros
Variable types: 0 continuous, 149 integer (149 binary)

Root relaxation: objective 3.600000e+02, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     360.0000000  360.00000  0.00%     -    0s

Exp